# Generating Bash Code with Granite Code Models

> **NOTE:** This recipe assumes you are working on a Linux, MacOS, or other UNIX-compatible system. While we haven't tested on Windows, some of the examples may generate valid DOS or PowerShell output. See comments below.


## Setting Up

### Install dependencies

Granite Kitchen comes with a bundle of dependencies that are required for Granite Cookbook recipes. See the list of packages in its [`setup.py`](https://github.com/ibm-granite-community/granite-kitchen/blob/main/setup.py). 

In [1]:
!pip install git+https://github.com/ibm-granite-community/utils \
    langchain_community \
    replicate


  Cloning https://github.com/ibm-granite-community/utils to /private/var/folders/8t/m9m188_d0tb8szvfqlc20hfr0000gn/T/pip-req-build-kfidbz59
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /private/var/folders/8t/m9m188_d0tb8szvfqlc20hfr0000gn/T/pip-req-build-kfidbz59
  Resolved https://github.com/ibm-granite-community/utils to commit a5965f40db3950dd2a41f3ca62a2c34adcdc20d7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Select a model

Select a Granite Code model from the [`ibm-granite`](https://replicate.com/ibm-granite) org on Replicate. Here we use the Replicate Langchain client to connect to the model.

To connect to a model on a provider other than Replicate, substitute this code cell with one from the [LLM component recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).

In [2]:
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var

model = Replicate(
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
)

### Detect your operating system

We'll find it useful to determine the name of our operating system and use that string in queries. This is because shell commands sometimes have different options on Linux vs. MacOS, etc. We'll write our queries so they take this difference into account. We can use a helper function to determine your OS. Note that `platform.system()` returns `Windows` on Windows system.

> **TIPS:** If you are using MacOS, you can install Linux-compatible versions of many commands. Consider these two options:
> * Install GNU Coreutils on a Mac. See [these instructions](https://superuser.com/questions/476575/replace-os-xs-shell-commands-with-the-linux-versions).
> * Install [HomeBrew](https://brew.sh/) and use it to install Linux-compatible (and other) tools.

In [3]:
import platform

def os_name():
    os_name = platform.system()
    # It turns out, using "MacOS" is better than "Darwin", which is what gets returned on MacOS.
    # For all other cases, the returned value should be fine as is, so we map the result to the desired
    # name, but only for MacOS...
    name_map = {'Darwin': 'MacOS'}
    shell_map = {'Windows': 'DOS'} # On Windows and use Power Shell, change from `DOS` to `Power Shell`.
    # ... then pass the os_name value as the second arg, which is used as the default return value.
    # For the shell name, return `bash` by default. (You can change this to zsh, fish, etc.)
    return name_map.get(os_name, os_name), shell_map.get(os_name, 'bash')

my_os, my_shell = os_name()
print(f"My OS is {my_os}. My shell is {my_shell}.")

My OS is MacOS. My shell is bash.



## One-shot Prompting with Granite Code Models

In One-shot prompting, you provide the model with a question and no examples. The model will generate an answer given its training. Larger models tend to do better at this task.

Note how we add additional context to the user's input prompt, such as _"make sure you write code that works for _my_ system!"_ (We'll see another way to do this below.)

In [4]:
from textwrap import dedent

prompt = dedent(f"""\
    Show me a {my_shell} script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time
    for each file in the output. Make sure you generate {my_shell} code that is {my_os}-compatible!"""
)

response = model.invoke(prompt)
print(response)

Here is a bash script that should meet your requirements:

```bash
#!/bin/bash

# Get the current timestamp one week ago
one_week_ago=$(date -v -1w "+%Y-%m-%d %H:%M:%S")

# Find files modified within the last week and print their names along with their last modification time
find . -type f -newermt "$one_week_ago" -printf "%Y-%m-%d %H:%M:%S %p\n" | head -n 50
```

This script first calculates the timestamp for one week ago from the current time. Then it uses the `find` command to search for files (`-type f`) that have been modified more recently than the calculated timestamp (`-newermt`). The `-printf` option is used to format the output, showing the last modification time (`%Y-%m-%d %H:%M:%S`) and the file path (`%p`). The `head -n 50` command ensures that only the first 50 results are displayed.

This script is compatible with MacOS, as it uses the `date` command available in MacOS's default shell (Bash).


### Try the script

Remove any markdown formatting in the output and paste the commands generated into the next cell _**after the %%bash line shown**_. Also delete the `ls -l`, which is there to allow the cell to run without error if nothing is pasted there (e.g., in our CI/CD test system). So, for example, you might have something like the following:

```shell
%%bash
find dir -type d | do_something
...
```

The `%%bash` "magic" tells Jupyter to run the commands as a shell script instead of as Python code. You can omit lines like `#!/bin/bash` and keep or remove any comments `# this is a comment...`.

Does the script work? If not try running the query again. Also try modifying the query string. What difference do these steps make?

In [5]:
%%bash
ls -l

total 56
-rw-r--r--  1 pravinkedia  staff  24581 Dec 30 13:17 Text_to_Shell.ipynb


We explore execution of generated shell code in the next recipe we recommend you study after this one, [../Text_to_Shell_Exec](../Text_to_Shell_Exec/Text_to_Shell_Exec.ipynb).

## Few-shot Prompting with Granite Code Models

In few-shot prompting, you provide the model with a question and some examples. The model will generate an answer given its training. The additional examples help the model zero in on a pattern, which may be required for smaller models to perform well at this task.

### Provide a list of Q&A examples


One of the examples uses the `stat` command, which requires different syntax for Linux vs. MacOS systems.

> **NOTE:** If you are using a Windows system, try changing the "answers" in the `examples` cell to be valid Power Shell or DOS commands. You can ignore the `stat_flags` in the next cell.

In [6]:
stat_flags = '-c "%y %n" {}'
if my_os == 'MacOS':
    stat_flags = '-f "%m %N" {}'
print(f"The 'stat' flags for my OS \'{my_os}\' and shell \'{my_shell}\' are \'{stat_flags}\'")

The 'stat' flags for my OS 'MacOS' and shell 'bash' are '-f "%m %N" {}'


In [7]:
examples = [
    {
        "question": "Recursively find files that match '*.js', and filter out files with 'excludeddir' in their paths.",
        "answer": "find . -name '*.js' | grep -v excludeddir",
    },
    {
        "question": "Dump \"a0b\" as hexadecimal bytes.",
        "answer": "printf \"a0b\" | od -tx1",
    },
    {
        "question": "Create a tar ball of all pdf files in the current folder and any subdirectories.",
        "answer": "find . -name '*.pdf' | xargs tar czvf pdf.tar",
    },
    {
        "question": "Sort all files and directories in the current directory, but no subdirectories, according to modification time, and print only the seven most recently modified items.",
        "answer": f"find . -maxdepth 1 -exec stat {stat_flags} '\' | sort -n -r | tail -n 7",
    },
    {
        "question": "Find all the empty directories in and under the current directory.",
        "answer": "find . -type d -empty",
    },
]

<>:16: SyntaxWarning: invalid escape sequence '\;'
<>:16: SyntaxWarning: invalid escape sequence '\;'
/var/folders/8t/m9m188_d0tb8szvfqlc20hfr0000gn/T/ipykernel_46078/866531455.py:16: SyntaxWarning: invalid escape sequence '\;'
  "answer": f"find . -maxdepth 1 -exec stat {stat_flags} \; | sort -n -r | tail -n 7",


### Assemble the prompt template

Here we build up a chat prompt template from messages. See the [Langchain docs](https://python.langchain.com/docs/how_to/few_shot_examples_chat/#fixed-examples) for more details.

In [8]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

chat_template = ChatPromptTemplate.from_messages(
    [
        few_shot_prompt,
        ("human", "{question}"),
    ]
)

print(chat_template.input_variables)


['question']


### View the completed prompt

Create a prompt and inspect the fully-interpolated chat template. Alternating Human/AI messages create a structure that the model will follow.

In [9]:
# Construct a prompt with no tabs and newlines.
prompt = dedent(f"""\
    Show me a {my_shell} script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time
    for each file in the output."""
).replace("\n", "")

print(chat_template.format(question=prompt))

Human: Recursively find files that match '*.js', and filter out files with 'excludeddir' in their paths.
AI: find . -name '*.js' | grep -v excludeddir
Human: Dump "a0b" as hexadecimal bytes.
AI: printf "a0b" | od -tx1
Human: Create a tar ball of all pdf files in the current folder and any subdirectories.
AI: find . -name '*.pdf' | xargs tar czvf pdf.tar
Human: Sort all files and directories in the current directory, but no subdirectories, according to modification time, and print only the seven most recently modified items.
AI: find . -maxdepth 1 -exec stat -f "%m %N" {} \; | sort -n -r | tail -n 7
Human: Find all the empty directories in and under the current directory.
AI: find . -type d -empty
Human: Show me a bash script to print the first 50 files found under the current working directorythat have been modified within the last week. Make sure you show the last modification timefor each file in the output.


### Run the model

In [10]:
chain = chat_template | model
response = chain.invoke({"question": prompt})
print(response)

Here is a bash script that should accomplish the task:

```bash
#!/bin/bash

# Find files modified in the last week and print their names along with modification times
find . -type f -newerwt -printf "%T+ %p\n" | head -50
```

This script works as follows:

- `find . -type f -newerwt`: This command finds all files (`-type f`) in the current directory (`.`) that have been modified or created (`-newerwt`) more recently than the timestamp of the search warrant (i.e., now).
- `-printf "%T+ %p\n"`: This option tells `find` to print the modification time (`%T+`) and the file path (`%p`) for each matching file, followed by a newline character (`\n`).
- `head -50`: This command limits the output to the first 50 lines.

Please note that you may need to make the script executable with the command `chmod +x script.sh` before running it. Replace `script.sh` with the actual name of your script file.


## Adding a System Prompt

Finally, a _system prompt_ is the preferred way to provide additional instructions and clarity about the context for a task, especially when this same information applies for _all_ user queries in the application. When you are building an AI-enabled application for a set of use cases, you will probably spend a lot of time refining the system prompt to maximize the quality of the results!

Here we define a system prompt to let the model know what we expect from it, including the expected language and os compatibility. To separate the system prompt from the user prompt, we use a chat template, which separates the prompt into messages and allows us to indicate the role of the speaker.


In [11]:
from langchain_core.messages import SystemMessage

system_prompt = SystemMessage(content=dedent(f"""\
    You are a helpful software engineer. You write clear, concise, well-commented code. Do not output anything but the code.
    Make sure you only generate {my_shell} code that is {my_os}-compatible!"""
))

chat_template_with_system_message = ChatPromptTemplate.from_messages(
    [
        system_prompt,
        few_shot_prompt,
        ("human", "{question}"),
    ]
)

print(chat_template_with_system_message.input_variables)

['question']


### Run the model

Note that the model has removed its chatter from around the code block.

In [12]:
chain_sys = chat_template_with_system_message | model
response = chain_sys.invoke({"question": prompt})
print(response)

Here is a bash script that will print the first 50 files found under the current working directory that have been modified within the last week, along with their last modification times:

```bash
#!/bin/bash

# Find files modified within the last week and print their names and modification times
find . -type f -newerwt -print0 | head -z -n 50 | while IFS= read -r -d '' file; do
    echo "File: $file"
    echo "Last Modified: $(date -r $file +'%Y-%m-%d %H:%M:%S')"
    echo ""
done
```

This script uses the `find` command to locate files modified within the last week (`-newerwt`). The `-print0` option is used to separate file names with null characters instead of newlines, which allows the script to handle file names with spaces or newlines. The `head -z -n 50` command limits the output to the first 50 files. The `while` loop reads each file name, prints the file name, and then uses `date -r` to display the last modification time in the desired format.


### For further study
- Try different queries. Test them to see if the model generated them correctly.
- Try invoking the model chain several times. How do the responses change from one invocation to the next?
- Try adding more examples to the `examples` string or modifying the ones shown. Does this affect the outputs?